In [5]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
adult = fetch_ucirepo(id=2) 
  
# data (as pandas dataframes) 
X = adult.data.features 
y = adult.data.targets 
  
df = X


In [22]:
import pandas as pd
import numpy as np

df: pd.DataFrame = X

print(df.shape)
df = df[~df.eq("?").any(axis=1)]
df.dropna()
print(df.shape)

df.to_csv("adults.csv", index=False)

# Identify categorical and numerical columns
categorical_cols = df.select_dtypes(include=['object']).columns
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns


df = pd.get_dummies(df, columns=categorical_cols)

cat_cols = df.select_dtypes(include=['object', 'category']).columns
cat_cols

(48842, 14)
(46443, 14)


Index([], dtype='object')

In [30]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
print(df)

# pca = PCA()  # Retain 95% of variance
# X_pca = pca.fit_transform(df)
# X_pca.shape
# print(X_pca)

# tsne = TSNE(n_components=2, perplexity=30)
# tsne_result = tsne.fit_transform(X_pca)  # shape: (n_samples, 2)

# # Optionally, if you have labels
# plt.scatter(tsne_result[:, 0], tsne_result[:, 1])
# plt.title("t-SNE Visualization")
# plt.show()

       age  fnlwgt  education-num  capital-gain  capital-loss  hours-per-week  \
0       39   77516             13          2174             0              40   
1       50   83311             13             0             0              13   
2       38  215646              9             0             0              40   
3       53  234721              7             0             0              40   
4       28  338409             13             0             0              40   
...    ...     ...            ...           ...           ...             ...   
48837   39  215419             13             0             0              36   
48838   64  321403              9             0             0              40   
48839   38  374983             13             0             0              50   
48840   44   83891             13          5455             0              40   
48841   35  182148             13             0             0              60   

       workclass_Federal-go